In [1]:
import sys
from pathlib import Path
from typing import List, Dict

from loguru import logger
from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import udf, col, when
import geopy.distance
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as F
from geopy.distance import great_circle
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit, struct
from pyspark.sql.functions import udf
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import pow, col, log
from pyspark.sql.functions import *
from pyspark.sql import Window
import pyspark.sql.functions as F
import geopandas as gpd


In [2]:
CONF_LOG_PREFIX = 'CONFLOG'
FLST_LOG_PREFIX = 'FLSTLOG'
GEO_LOG_PREFIX = 'GEOLOG'
LOS_LOG_PREFIX = 'LOSLOG'
REG_LOG_PREFIX = 'REGLOG'
LOADING_PATH = '../output'
DATAFRAMES_NAMES = [CONF_LOG_PREFIX, FLST_LOG_PREFIX, GEO_LOG_PREFIX, LOS_LOG_PREFIX, REG_LOG_PREFIX]

Give access to the constants that defines

In [3]:
sys.path.append(str(Path(Path().absolute().parent, 'platform_code')))
from schemas.tables_attributes import *
from utils.config import settings

In [4]:
def load_dataframes(files_names: List[str], loading_path: str, spark: SparkSession) -> Dict[str, DataFrame]:
    """ Loads the dataframes which macht the file names passed by arguments.
    The method read from the config the path were to read the files, which
    matches the folder where the files are saved in `save_dataframes_dict()`.

    :param files_names: list of the names of the files.
    :param loading_path: path were the files are saved.
    :param spark: spark session.
    :return: dictionary with the dataframes loaded from the files, with the
     file name as key.
    """
    dataframes = dict()

    for file_name in files_names:
        file_path = Path(loading_path, f'{file_name.lower()}.parquet')
        logger.info('Loading dataframe from `{}`.', file_path)
        df = spark.read.parquet(str(file_path))
        dataframes[file_name] = df

    return dataframes

In [5]:
spark = SparkSession.builder.appName('Notebook').getOrCreate()

In [6]:
input_dataframes = load_dataframes(DATAFRAMES_NAMES, LOADING_PATH, spark)

2022-04-04 16:42:40.805 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\conflog.parquet`.
2022-04-04 16:42:43.794 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\flstlog.parquet`.
2022-04-04 16:42:43.921 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\geolog.parquet`.
2022-04-04 16:42:44.003 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\loslog.parquet`.
2022-04-04 16:42:44.092 | INFO     | __main__:load_dataframes:16 - Loading dataframe from `..\output\reglog.parquet`.


In [8]:
@udf
def get_coordinates_distance(origin_latitude: float, origin_longitude: float,
                             destination_latitude: float, destination_longitude: float) -> float:
    """ Calculates the distance in meters between two world coordinates.

    :param origin_latitude: origin latitude point.
    :param origin_longitude: origin longitude point.
    :param destination_latitude: destination latitude point.
    :param destination_longitude: destination longitude point.
    :return: distance in meters.
    """
    origin_tuple = (origin_latitude, origin_longitude)
    destination_tuple = (destination_latitude, destination_longitude)
    # TODO: direct distance calculation (in meters) between two points, is this approach correct?
    return geopy.distance.distance(origin_tuple, destination_tuple).m

# ENV-2: Weighted average altitude
Average flight level weighed by the length flown at each flight level.

In [ ]:
dataframe = input_dataframes[REG_LOG_PREFIX]

First, we check the log for a given drone in a given scenario.

In [ ]:
#dataframe.where(col(SCENARIO_NAME) == '1_very_low_40_8_R2').where(col(ACID) == 'D1').orderBy(SIMULATION_TIME, ACID).show()

Create a column with the next coordinates system for the same drone

In [ ]:
window = Window.partitionBy(SCENARIO_NAME, ACID).orderBy(SIMULATION_TIME)
next_step = dataframe.withColumn("NEXT_LATITUDE",lag(LATITUDE, -1).over(window)).withColumn("NEXT_LONGITUDE",lag(LONGITUDE, -1).over(window)).withColumn("NEXT_ALTITUDE",lag(ALTITUDE, -1).over(window))

In [ ]:
#next_step.where(col(SCENARIO_NAME) == '1_very_low_40_8_R2').where(col(ACID) == 'D1').show(50)

In [ ]:
#Remove rows with NEXT_LATITUDE and NEXT_LONGITUDE null (they are the rows of separation between scenarios)
dataframe = next_step.na.drop(subset=["NEXT_LATITUDE","NEXT_LONGITUDE"])
#Check it
dataframe.filter(col("NEXT_LATITUDE").isNull() | col("NEXT_LATITUDE").isNull()).show()

In [ ]:
dataframe = dataframe.withColumn("SEGMENT_LENGTH",get_coordinates_distance(LATITUDE, LONGITUDE, "NEXT_LATITUDE", "NEXT_LONGITUDE")).withColumn("SEGMENT_ALTITUDE", (col(ALTITUDE)+col("NEXT_ALTITUDE"))/2)
dataframe = dataframe.withColumn("SEGMENT_WEIGHT", col("SEGMENT_ALTITUDE") * col("SEGMENT_LENGTH"))

In [ ]:
#dataframe.select(REG_ID, SCENARIO_NAME, ACID, "SEGMENT_LENGTH", "SEGMENT_ALTITUDE", "SEGMENT_WEIGHT").show()

In [ ]:
dataframe.groupby(SCENARIO_NAME).agg(sum(col("SEGMENT_WEIGHT")), sum(col("SEGMENT_LENGTH"))).withColumn(ENV2, col("sum(SEGMENT_WEIGHT)")/col("sum(SEGMENT_LENGTH)")).show()

In [9]:
#ENV4
dataframe4 = input_dataframes[REG_LOG_PREFIX]
dataframe4 = dataframe4.groupby(SCENARIO_NAME, ACID).agg(F.max(ALTITUDE).alias("MAX_ALTITUDE"), F.min(ALTITUDE).alias("MIN_ALTITUDE"))
dataframe4 = dataframe4.withColumn("DIFF_ALTITUDE", col("MAX_ALTITUDE") - col("MIN_ALTITUDE"))
avg_delay = dataframe4.select(mean("DIFF_ALTITUDE").alias("MEAN_DIFF_ALTITUDE"))
dataframe4 = dataframe4.join(avg_delay, how='outer')
dataframe4.show()

dataframe4 = dataframe4.withColumn(ENV4, col("DIFF_ALTITUDE")/col("MEAN_DIFF_ALTITUDE"))
dataframe4 = dataframe4.select(SCENARIO_NAME, ACID, ENV4)
dataframe4.show()
# dataframe5 = dataframe4.groupby(SCENARIO_NAME).agg(F.max('MAX_ALTITUDE').alias("MAX_ALTITUDE_SCN"), F.min('MIN_ALTITUDE').alias("MIN_ALTITUDE_SCN"))
# dataframe5.show()


+------------------+-----+------------------+------------------+------------------+------------------+
|          Scenario| ACID|      MAX_ALTITUDE|      MIN_ALTITUDE|     DIFF_ALTITUDE|MEAN_DIFF_ALTITUDE|
+------------------+-----+------------------+------------------+------------------+------------------+
|1_very_low_40_8_R2|D1351|            82.296|   41.389389999312|40.906610000688005|   78.248320994009|
|1_very_low_40_8_R2|D1652|   132.04138999848|             9.144|122.89738999848001|   78.248320994009|
|1_very_low_40_8_R2|D1740|           118.872|12.618610000728001|  106.253389999272|   78.248320994009|
|1_very_low_40_8_R2|D2171|  132.346610000688|             9.144|  123.202610000688|   78.248320994009|
|1_very_low_40_8_R2|D2311|  102.414609999936|             9.144|   93.270609999936|   78.248320994009|
|1_very_low_40_8_R2|D2366|           146.304|1.3893899994720003|  144.914610000528|   78.248320994009|
|1_very_low_40_8_R2|D2395|            82.296|            18.288| 64.00800

In [ ]:
from pyspark.sql.functions import udf
def get_coordinates_distance(origin_latitude: float, origin_longitude: float,
                             destination_latitude: float, destination_longitude: float) -> float:
    """ Calculates the distance in meters between two world coordinates.

    :param origin_latitude: origin latitude point.
    :param origin_longitude: origin longitude point.
    :param destination_latitude: destination latitude point.
    :param destination_longitude: destination longitude point.
    :return: distance in meters.
    """
    origin_tuple = (origin_latitude, origin_longitude)
    destination_tuple = (destination_latitude, destination_longitude)
    # TODO: direct distance calculation (in meters) between two points, is this approach correct?
    return geopy.distance.distance(origin_tuple, destination_tuple).m


@udf
def in_circle (x_center, y_center, x, y, radius):
    return get_coordinates_distance(x_center, y_center, x, y) <= radius

@udf("double")
def great_circle_udf(x, y):
    return great_circle(x, y).kilometers

In [ ]:

print(settings.x_center)
print(settings.time_roi)
print(settings.radius_roi)

In [ ]:
dataframe3.show()

In [ ]:
def loadEnv3points(geojson):
    return gpd.read_file(geojson)[0:10] #only first 100 rows

In [ ]:
df = spark.read.json("env3_points.json")

In [ ]:
print(df)

In [ ]:
for d in df_points.iterrows():
    print(d)

In [ ]:
#ENV3
dataframe3 = input_dataframes[REG_LOG_PREFIX]
udf_func = udf(great_circle_udf,DoubleType()) #Creating a 'User Defined Function' to calculate distance between two points.



In [ ]:
#ENV3
dataframe3 = input_dataframes[REG_LOG_PREFIX]

#dataframe3.printSchema() 
#dataframe3.show()
df_points = loadEnv3points(settings.geojson.path)

for i,(x, y) in enumerate(zip(df_points.geometry.x, df_points.geometry.y)):
    point = struct(lit(y), lit(x))
    
    aux_dataframe = dataframe3.filter(col(SIMULATION_TIME) == settings.env3.time_roi)
    aux_dataframe = aux_dataframe.withColumn("distance", great_circle_udf(point, struct(col(LATITUDE), col(LONGITUDE))))
    aux_dataframe = aux_dataframe.filter((col("distance") <= 16))
    aux_dataframe = aux_dataframe.withColumn("sound_intensity", 1/(pow((col("distance")/settings.flight_altitude.lowest), 2)))
    aux_dataframe = aux_dataframe.groupby(SCENARIO_NAME).agg(sum("sound_intensity").alias(f"ENV3_p{i}"))
    
    
    if(i==0):
        final_dataframe = aux_dataframe
    else:
        final_dataframe = final_dataframe.join(aux_dataframe, SCENARIO_NAME)
        

    
    


In [ ]:
final_dataframe.show()


In [ ]:


point = struct(lit(x_center), lit(y_center))
udf_func = udf(great_circle_udf,DoubleType()) #Creating a 'User Defined Function' to calculate distance between two points.
dataframe3 = dataframe3.withColumn("distance", udf_func(point, struct(col(LATITUDE), col(LONGITUDE)))).filter((col("distance") <= radius_roi) & (col(ALTITUDE)<= altitude_roi) & (col(SIMULATION_TIME) == time_roi)) #Creating column "distance" based on function 'get_distance'
#dataframe3 = dataframe3.withColumn("noise_level", 10*log(1/pow(col("distance"),2)))
dataframe3 = dataframe3.withColumn("noise_level", log10(1/pow(col("distance"),2)))
dataframe3.show()

In [ ]:
dataframe3.groupBy(SCENARIO_NAME).agg(sum("noise_level")).show()

In [ ]:
dataframe3.show()

In [ ]:



dataframe = input_dataframes[REG_LOG_PREFIX]

point = struct(lit(settings.x_center), lit(settings.y_center))

# TODO: ? Define formula for the sound depending on the distance to the point.
# TODO: ? How many points and how to define.
return dataframe.filter(col(SIMULATION_TIME) == settings.time_roi) \
    .withColumn("distance", great_circle_udf(point, struct(col(LATITUDE), col(LONGITUDE)))) \
    .filter((col("distance") <= settings.radius_roi) & (col(ALTITUDE) <= settings.altitude_roi)) \
    .withColumn("noise_level", log10(1 / pow(col("distance"), 2))) \
    .groupBy(SCENARIO_NAME).agg(sum("noise_level").alias(ENV3))